In [33]:
import os
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from scipy import spatial
p_raw = '/project/nsaru/raw/PySlake/'
p_work = '/project/nsaru/work/PySlake/'

In [34]:
def within(circle,point):
    if np.sqrt(np.dot(circle[0:2]-point,circle[0:2]-point))<circle[2]:
        return True
    else:
        return False

In [35]:
def nearest_cell(circles,centroids):
    cells_xy = circles[0][:,0:2]
    aggrs_xy = np.array(centroids)
    d = spatial.distance_matrix(cells_xy,aggrs_xy)
    cells_idx = []
    for id in range(d.shape[1]):
        i = np.argmin(d[:,id])
        if within(circles[0][i,:],aggrs_xy[id,:]):
            cells_idx.append(np.array([i,id]))
        
    return cells_idx

In [ ]:
def cell_ids(circles,cell_labels):
    return
    

In [28]:
samples = []
times = []
cells = []
all_matches = []
all_areas = []
for d in os.listdir(p_raw):
    samples.append(d)
    t=[]
    matches = []
    areas = []
    for f in os.listdir(os.path.join(p_raw,d)):
        if '.csv' in f:
            if f!='ImageInfo.csv':
                cell_labels = pd.read_csv(os.path.join(p_raw,d,f))
    j = 0
    p_img = os.path.join(p_raw,d,'Images')
    if not os.path.isdir(p_img): 
        print(p_img)
        p_img = os.path.join(p_raw,d,'images')

    for f in os.listdir(p_img):
        t.append(os.path.getctime(os.path.join(p_img,f)))
        img = cv2.imread(os.path.join(p_img,f))
        b,g,r = cv2.split(img) 
        h,s,v = cv2.split(cv2.cvtColor(img,cv2.COLOR_BGR2HSV))
        if j==0:
            circles = cv2.HoughCircles(b,cv2.HOUGH_GRADIENT,1,175,param1=50,param2=30,minRadius=110,maxRadius=140)
            circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            # draw the outer circle
            cv2.circle(img,(i[0],i[1]),i[2],(255,0,0),5)

            #plt.subplot(211)
            #plt.imshow(img)
        ret, thresh = cv2.threshold(v,120,255,cv2.THRESH_BINARY_INV) 
        contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
        centers = []
        A = []
        for c in contours:
            # compute the center of the contour
            M = cv2.moments(c)
            if M["m00"]>0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                centers.append(np.array([cX,cY]))
                A.append(M["m00"])
        # draw contours on the original image
        cv2.drawContours(image=img, contours=contours, contourIdx=-1, color=(255, 0, 0), thickness=5, lineType=cv2.LINE_AA)

        #plt.subplot(212)
        #plt.imshow(img)
        #plt.show()
        cells_idx = nearest_cell(circles,centers)
        matches.append(cells_idx)
        areas.append(A)
        j = j+1
    
    all_matches.append(matches)
    all_areas.append(areas)
    times.append(t)

/project/nsaru/raw/PySlake/Kelly P24_Time1_Depth2/Images


In [32]:
all_areas[0][0]


[2.5,
 55.5,
 341.5,
 23.5,
 0.0,
 13.5,
 0.0,
 110.5,
 2.0,
 0.0,
 6047.0,
 3609.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.0,
 168.5,
 4.0,
 4.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.5,
 0.0,
 0.0,
 1.5,
 0.0,
 0.0,
 9.5,
 4420.0,
 0.0,
 39.5,
 125.5,
 4.0,
 2.0,
 7.0,
 2.0,
 1.0,
 3.5,
 0.5,
 2893.0,
 1.0,
 0.0,
 1064.0,
 167.0,
 2.0,
 8.5,
 0.0,
 4.0,
 3218.5,
 6.0,
 2.0,
 17.0,
 6495.0,
 11.5,
 8.5,
 1.0,
 5.0,
 0.0,
 0.5,
 0.0,
 2224.0,
 0.0,
 5658.0,
 2.0,
 2.0,
 17.5,
 10.0,
 8.0,
 0.0,
 5.5,
 4.0,
 5635.5,
 4820.5,
 2.0,
 5864.5,
 6534.5,
 14.0,
 5.0,
 18.5,
 0.0,
 28.5,
 175.0,
 98.5,
 6.5,
 1.0]